# Use Customized Notebook Template (CLAIMS_RESERVE_Template)

In [1]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import statsmodels
import matplotlib.pyplot as plt
import seaborn as sns

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from fosforio import snowflake
from snowflake.snowpark.functions import col, lit, when, year, to_date


Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

table_name = 'AUTO_INSURANCE_CLAIMS_DATA'
sf_df = my_session.sql("select * from {}".format(table_name))


In [3]:
# 1. Filter data for rows where INCIDENT_SEVERITY is 'Major Damage' and POLICY_STATE is 'CT'
filtered_df = sf_df.filter((col("INCIDENT_SEVERITY") == lit("Major Damage")) & (col("POLICY_STATE") == lit("CT")))